#### Install all the required packages using pip first

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install psycopg2-binary

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
!pip install pyap

##### <b>Customization steps for pyap<b>  
1)  Login into Terminal  
2)  Install vim  
    a)  apt-get update  
    b)  apt-get install vim  
3)  vi data.py in /opt/conda/lib/python3.7/site-packages/pyap/source_US  
4)  Delete all contents (1,$d) and refresh the file with /root/scripts/data.py (r: /root/scripts/data.py)

In [ ]:
import re
import os
import glob
from pathlib import Path
import psycopg2
import pyap
import spacy
# from flair.data import Sentence
# from flair.models import SequenceTagger
# import string


In [ ]:
NER = spacy.load("en_core_web_trf", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])

#### Define All Regular Expresssions  

In [ ]:
#----------------------------------- Define Regular Expresssions -----------------------------------#

REMOVE_CTRL_CHAR = re.compile('[^\x00-\x7F]+')
URL_REGEX = re.compile(
    r"(?:^|(?<![\w\/\.]))"
    r"(?:(?:https?:\/\/|ftp:\/\/|www\d{0,3}\.))"
    r"(?:\S+(?::\S*)?@)?" r"(?:"
    r"(?!(?:10|127)(?:\.\d{1,3}){3})"
    r"(?!(?:169\.254|192\.168)(?:\.\d{1,3}){2})"
    r"(?!172\.(?:1[6-9]|2\d|3[0-1])(?:\.\d{1,3}){2})"
    r"(?:[1-9]\d?|1\d\d|2[01]\d|22[0-3])"
    r"(?:\.(?:1?\d{1,2}|2[0-4]\d|25[0-5])){2}"
    r"(?:\.(?:[1-9]\d?|1\d\d|2[0-4]\d|25[0-4]))"
    r"|"
    r"(?:(?:[a-z\\u00a1-\\uffff0-9]-?)*[a-z\\u00a1-\\uffff0-9]+)"
    r"(?:\.(?:[a-z\\u00a1-\\uffff0-9]-?)*[a-z\\u00a1-\\uffff0-9]+)*"
    r"(?:\.(?:[a-z\\u00a1-\\uffff]{2,}))" r"|" r"(?:(localhost))" r")"
    r"(?::\d{2,5})?"
    r"(?:\/[^\)\]\}\s]*)?",
    flags=re.UNICODE | re.IGNORECASE | re.MULTILINE
)

# email_pattern = re.compile(r"([a-z0-9!#$%&'*+\/=?^_`{|.}~-]+@(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)", flags=re.UNICODE | re.IGNORECASE | re.MULTILINE)

email_pattern = re.compile(r"[\w\.-]+@[\w\.-]+\.\w+", flags=re.UNICODE | re.IGNORECASE | re.MULTILINE)

suite_pattern = re.compile(r"(?:suite\s*\d{1,5}$)", flags=re.UNICODE | re.IGNORECASE | re.MULTILINE)


#### Declare Matching Patterns  

In [ ]:
#----------------------------------- Define Match Words -------------------------------------#

ooo_matches = ("will be on pto", "m out of office", "m out of the office", "ll be out of office", "ll be out of the office", 
               "m currently out of office", "m currently out of the office")

undelivered_matches = ("was undeliverable.", "subject: undeliverable:")

volunteer_matches = ( "volunteers needed! ", "rds listening session – " )

sal_st_matches = ("rds:", "hello", "good ", "dear", "hi", "to whom it may concern")

sal_end_matches = ("sincerely,", "thanks", "thank you", "thanks in advance", "regards", "best", "kind regards", 
                   "kindest regards,", "kindly,", "blessings,", "sent from ")

addr_end_matches = (" st.", " street", " road", " plaza", " place", " dr.", " dr", " blvd", " blvd.", " floor", " lane", " ave", 
                    " avenue", " ct.", " parkway", " highway", " boulevard", " park")

sig_start_matches = ("office", "ph:", "fax", "mobile", "work:")

suppref_matches  = ("PersonEmailAddress:", "PersonName:", "PersonUserName:", "PersonID:", "PlanSponsorID:", "ReferenceNumber:", 
                     "ApplicationID:", "Priority:", "URL:", "SLATime:")

po_box_matches = ("p.o. box", "p.o box", "po box")

ignore_matches = ("name: ", "your name", "application number:", "email:", "url:", "sent:", "to:", "cc:", "phone", 
                  "m ", "organization name: ", "plan sponsor id: ", "applicationid: ", "plan year start date:", 
                  "plan year end date:", "please note: this email", "please refer to the rds user", 
                  "if you need further assistance with this", "important: when a user enters login", "action needed: ", 
                  "rds vendor name",  "rds vendor id ", "job title", "telephone number", "email address:", "technical contact")

matches = ["original message", "delete this line", "external sender", "external mail", "content is safe", "unless you recognize"]

replace_list = [ "' rds center", " csd ", " re ", " fw ", "ph: ", "fax", "(mobile)", "(work)", "(office)", "(fax)", "samrat", " rick", 
                " est", " 's", " the ", "mozilla", "cms", "rds" ]

nonEnglish_matches = ("avis :", "title avis :", "anmerkung:", "upozornení:") 

suppref_replace_list = [ "QuestionSubCategoryDesc:", "TopicCategoryDesc:", "UserQuestionText:" ]

org_ignore_matches = ( "acct", "account", "active directory", "actuary", "application", "appeal", "authorized", "auth rep", "authenticat", 
                      "benefit", "client/sponsor", "chrome", "cost", "data entry", "designee", "drug subsidy", "factor", "file", 
                      "guideds", "help center", "ira", "login", "mfa", "option", "payment", "pending", "plan", "program", "registration", 
                      "retiree", "report", "setup", "sponsor", "submission", "recon", "representative", "user", "website" ) 

org_list = [ "non", "nonprofit", "operations", "pass", "pending", "plans", "rds appeals", "rds / ira", "rds login", "rds - mfa", "rds plan", 
            "rds program", "rds program website", "rds secure", "rds secure website", "rds secure website to rds center", "rds site", 
           "rds team", "rds user", "recon", "reconciliation", "reconciliation cost reporting", "remove vendor",
           "resource library", "retail accounting", "retiree", "retiree drug subsidy", "retiree management", "retiree medical", 
            "retirement benefits & compliance", "review & submit application", "rx benefits", "secure", "secure transport", "secure website", 
           "teams", "action for reconciliation" ]


#### Define all functions next

In [ ]:
#------------------------------ Function to print errors from PostgreSQL ------------------------------#

def print_psycopg2_exception(err):
    err_type, err_obj, traceback = sys.exc_info()
    line_num = traceback.tb_lineno
    print ("\npsycopg2 ERROR:", err, "on line number:", line_num)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type)
    print ("\nextensions.Diagnostics:", err.diag)
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")
    input("Press Enter to continue...")
    

In [ ]:
#-------------- Read PostgreSQL and get processed email keys into a list first -------------------------#

def read_postgres():
    records_in_pgres = []
    try:    
        cursor.execute("SELECT email_key FROM ml_extract_track_v2 WHERE extract_status IN ('C', 'P'); ")
        print(cursor.rowcount, "record(s) found in PostgreSQL", "\n")
        rows = cursor.fetchall()
        for row in rows:
            records_in_pgres.append(row[0])

    except Exception as err:
        print_psycopg2_exception(err)

    return records_in_pgres


In [ ]:
#------------------------------ Update PostgreSQL for the email_key -------------------------------------#

def upd_postgres_record(email_key, extract_status, ignore_status, response_status, email_type):
    upd_query = "UPDATE ml_extract_track_v2 SET "
    upd_query = upd_query + "extract_status = '" + extract_status + "', email_type = '" + email_type + "', "
    upd_query = upd_query + "ignore_status = '" + ignore_status + "', response_status = '" + response_status + "', "
    upd_query = upd_query + "last_update_date = now() WHERE email_key = '" + email_key + "' AND extract_status = 'E'; "
    # print(upd_query)

    # Execute SQL Command and insert the details to PostgreSQL
    try:
        cursor.execute(upd_query)
        conn.commit()
        count = cursor.rowcount
        print("   ", count, " record updated successfully\n")
        # cnt = cnt + 1

    except Exception as err:
        print_psycopg2_exception(err)
        conn.rollback()
        

In [2]:
def find_supp_ref_eom(trimmedTxt):
    finalBody = ""
    email_type = ""
    response_status = ""
    sal_st_found = False
    sal_end_found = False
    from_found = False
    fw_pattern = False
    lno = 0

    for w in suppref_replace_list:
            trimmedTxt = trimmedTxt.replace(w, "")
    
    lines = trimmedTxt.splitlines()
    for line in lines:
        lno = lno + 1
        
        if line.startswith(suppref_matches):
            continue
              
        lineLower = line.lower()
        l = len(line)
        word_cnt = len(line.split())

        if not (sal_st_found):
            if line.endswith((",", ":", "-", "!")) and word_cnt < 4:
                sal_st_found = True
                continue
                
            if lineLower.startswith(sal_st_matches):
                sal_st_found = True
                if word_cnt < 6:
                    continue
                else:
                    # Look for comma, hyphen or colon
                    comma_loc = lineLower.find(",")
                    if comma_loc == -1: comma_loc = lineLower.find(":")
                    if comma_loc == -1: comma_loc = lineLower.find("-")
                    if comma_loc != -1:
                        finalBody = finalBody + line[comma_loc+1:]
                        continue

        # Look for "Sincerely", "Thanks" etc. If found, write the file
        if lineLower.startswith(sal_end_matches):
            if word_cnt < 6 and lno > 3:
                sal_end_found = True
                # Get out of FOR LOOP now
                break

        if lineLower.endswith(addr_end_matches) and word_cnt < 8:
            addr_found = True
            continue

        if lineLower.endswith(sig_start_matches) and word_cnt < 5:
            continue
            
        if lineLower.startswith(po_box_matches):
            continue

        if (lineLower.find("p.o. box") != -1 or lineLower.find("p.o box") != -1 or lineLower.find("po box") != -1) and word_cnt < 15:
            continue

        # See which mataches and cleanups are applicable 5/29

        if lineLower.startswith(ignore_matches):
            continue
        
        finalBody = finalBody + line + "\n"
    
    return finalBody, sal_end_found

In [3]:
#------------------------------- Cleanup for RDS Support Request Ref -----------------------------------#

def find_end_of_message(omhStripped):
    finalBody = ""
    email_type = ""
    response_status = ""
    sal_st_found = False
    sal_end_found = False
    from_found = False
    fw_pattern = False
    lno = 0

    lines = omhStripped.splitlines()
    for line in lines:
        lno = lno + 1
        if lno == 1:
            if line.startswith("re: ") and re.search('\d', line):
                email_type = "RESPONSE"
                response_status = 'R'
                loc = line.rfind("re: ")
                locf = line.rfind("fw: ")
                if loc > locf:
                    finalBody = finalBody + line[loc+3:] + "\n"
                else:
                    finalBody = finalBody + line[locf+3:] + "\n"
                
                continue

            if line.startswith("fw:"):
                # it is a whole new ball game! Read until "Thanks" or name
                # But skip lines starting with "From", "Sent", "To", email-id etc. Include "Subject"
                fw_pattern = True
                email_type = "FORWARD"
                response_status = 'F'
                from_count = 0
            
            if line.startswith("=?utf-8?"):
                continue

            finalBody = finalBody + line + ". "
            continue
        
        lineLower = line.lower()
        l = len(line)
        word_cnt = len(line.split())

        # Looks for Salutations like "hello", "good ", "dear " or "hi"
        if lno < 4 and lno > 1 and sal_st_found is False:
            if line.endswith((",", ":", "-", "!")) and word_cnt < 4:
                sal_st_found = True
                continue
            if lineLower.startswith(sal_st_matches):
                sal_st_found = True
                if word_cnt < 6:
                    continue
                else:
                    # Look for comma, hyphen or colon
                    comma_loc = lineLower.find(",")
                    if comma_loc == -1: comma_loc = lineLower.find(":")
                    if comma_loc == -1: comma_loc = lineLower.find("-")
                    if comma_loc == -1: comma_loc = lineLower.find("!")
                    if comma_loc != -1:
                        finalBody = finalBody + line[comma_loc+1:]
                        sal_st_found = True
                        continue
        
        if lno < 4 and line.endswith((",", ":", "-", "!")) and word_cnt < 3:
            sal_st_found = True
            continue
            
        # Look for "Sincerely", "Thanks" etc. If found, write the file
        if lineLower.startswith(sal_end_matches):
            if word_cnt < 6 and lno > 3:
                sal_end_found = True
                # Get out of FOR LOOP now
                break

        if lineLower.endswith(addr_end_matches) and word_cnt < 8:
            addr_found = True
            break

        if lineLower.endswith(sig_start_matches) and word_cnt < 5:
            addr_found = True
            continue
            
        if lineLower.startswith("from:"):
            if (fw_pattern):
                if from_count > 0:
                    break
                else:
                    from_count += 1
                    continue
            else:
                break

        if lineLower.startswith("on ") and lineLower.endswith("wrote:"):
            continue

        # This line is followed by "--Original Message--"
        if lineLower.startswith("---------------------------------------------"):
            break

        if lineLower.startswith("subject:"):
            finalBody = finalBody + line[8:]
            continue
        
        if lineLower.startswith(po_box_matches):
            continue

        if (lineLower.find("p.o. box") != -1 or lineLower.find("p.o box") != -1 or lineLower.find("po box") != -1) and word_cnt < 15:
            continue
            
        if lineLower.startswith(ignore_matches):
            continue

        if lineLower.startswith("www.") and word_cnt < 3:
            continue
        
        finalBody = finalBody + line + "\n"
    
    return finalBody, email_type, response_status, sal_end_found
            

In [4]:
#-------------------------------- Final Cleanup before writing to file ----------------------------------#

def final_cleanup(finalBody, email_type, sal_end_found):

    org_list = ""
    emailStripped = re.sub(email_pattern, ' ', finalBody)
    trimmedTxt = re.sub(r' {2,}' , ' ', emailStripped)
    
    cleanedText = trimmedTxt
    if email_type == "SUPP_REF":
        for w in suppref_replace_list:
            cleanedText = cleanedText.replace(w, " ")
            
    name_found = False
    nameStripped = cleanedText
    text1= NER(cleanedText)
    for word in text1.ents:
        if word.label_ == "PERSON":
            if sal_end_found is True or email_type == "SUPP_REF":
                altered_text = nameStripped.replace(word.text, " ")
                nameStripped = altered_text
                continue
            else:
                loc = nameStripped.find("\n" + word.text)
                if loc != -1:
                    word_cnt = 100 # some big number
                    nloc = nameStripped.find("\n", loc+1)
                    if nloc != -1:
                        line = nameStripped[loc+1:nloc]
                        word_cnt = len(line.split())
                    if nloc == -1 or word_cnt < 9:
                        altered_text = nameStripped[0:loc]
                        nameStripped = altered_text
                        name_found = True
                        continue     
                else:
                    altered_text = nameStripped.replace(word.text, " ")
                    nameStripped = altered_text
                    continue
    
    # Limit the message at the address line
    addrStripped = nameStripped
    addresses = pyap.parse(nameStripped, country='US')
    if addresses:
        # print("found addresses")
        for address in addresses:
            str_addr = str(address)
            if sal_end_found is True or name_found is True or email_type == "SUPP_REF":
                altered_text = addrStripped.replace(str_addr, " ")
                addrStripped = altered_text
                continue
            else:
                loc = addrStripped.find("\n"+ str_addr)
                if loc != -1:
                    altered_text = addrStripped[0:loc]
                    addrStripped = altered_text
                    break
                else:
                    altered_text = addrStripped.replace(str_addr, " ")
                    addrStripped = altered_text
                    continue 
    else:
        if sal_end_found is False and name_found is False and email_type != "SUPP_REF":
            # print("No address found.. trying for suite")
            # Look for lines ending in "Suite #"
            match = re.search(suite_pattern, addrStripped)
            if match != None:
                # print("Match at index %s, %s" % (match.start(), match.end()))
                match_str = addrStripped[match.start():match.end()]
                loc = addrStripped.find(match_str)
                altered_text = addrStripped[0:loc]
                # print("match_str: ", match_str, "altered_text:", altered_text)
                nloc = altered_text.rfind("\n")
                addrStripped = altered_text[0:nloc]
                # print("Suite Found:", addrStripped, "\n")
    
    for word in text1.ents:
        if word.label_ == "GPE":
            altered_text = addrStripped.replace(word.text, " ")
            addrStripped = altered_text
            continue
        if word.label_ == "ORG":
            word_lower = word.text.lower()
            if any([x in word_lower for x in org_ignore_matches]):
                continue
            else:
                altered_text = addrStripped.replace(word.text, " ")
                addrStripped = altered_text
                # org_list = org_list + word.text + "\n"
                continue
    
    emailStripped = addrStripped.replace("’", "'")
    urlStripped = re.sub(URL_REGEX, ' ', emailStripped)
    urlStripped = REMOVE_CTRL_CHAR.sub(' ', urlStripped)
    
    replacedText = urlStripped.lower()    
    for w in replace_list:
        replacedText = replacedText.replace(w, " ")
            
    noDigitTxt = ''.join((x for x in replacedText if not x.isdigit()))
    noPuncTxt = re.sub(r"[^\w\s\'\’]", ' ', noDigitTxt)
    noOneCharTxt = ' '.join([word for word in noPuncTxt.split() if len(word)>1 ])
    cleanedText = noOneCharTxt
    
    cleanedTxt = re.sub(r' {2,}' , ' ', cleanedText)
    # print("Cleaned txt:\n", cleanedTxt, "\n", "ignore_status:", ignore_status, "response_status:", response_status, "email_type:", email_type)
    # input("Enter to continue..")
    
    return cleanedTxt, org_list
    

In [ ]:
# path = "/home/sagemaker-user/ml_orgtxt_emails/"
# out_cleanfile_folder = "/home/sagemaker-user/ml_cleaned_emails_r2/"
path = "/root/ml_orgtxt_emails/"
out_cleanfile_folder = "/root/ml_cleaned_emails/"
errorFile = "/root/orgs/error_files.lst"
# orgListFile = "/root/orgs/orgs.lst"

In [200]:
#------------------------------- Connect to PostgreSQL and read records ---------------------------------#

cursor.close()
conn.close()
conn = psycopg2.connect(database="YOUR_DATABASE", user='<YOUR_USER>', password='<YOUR_PASSWORD>', 
                        host='<DATABASE_HOST>', port= '5432')
cursor = conn.cursor()

records_in_pgres = read_postgres()

# Get sender information in a dictionary
sender_info = []
try: 
    cursor.execute("SELECT email_key, sender FROM ml_extract_track_v2 WHERE extract_status = 'E'; ")
    print(cursor.rowcount, "record(s) found in PostgreSQL", "\n")
    rows = cursor.fetchall()
    for row in rows:
        sender_info.append(
            { 'emailkey': row[0], 
             'sender': row[1] })

except Exception as err:
    print_psycopg2_exception(err)


13664 record(s) found in PostgreSQL 

2 record(s) found in PostgreSQL 



In [ ]:
#--------------------------------------- Main Program Starts Here ----------------------------------------#

cnt = 1
out_error_file = open(errorFile, "a")
# org_list_file = open(orgListFile, "a")

for filename in glob.glob(os.path.join(path, '*.txt')):

    # Initialize variables
    loc = 0 
    nloc = 0
    cidStripped = ""
    trimmedTxt = ""
    omhStripped = ""
    eomText = ""
    cleanedTxt = ""
    first_line = ""
    second_line = ""
    org_list = ""
    email_type = ""
    ignore_status = ""
    response_status = "" 
    sal_end_found = False 
     
    if (cnt > 1000): 
        break
    
    with open(filename, 'r') as f: 
        fileContent = f.read()
        content_lower = fileContent.lower()
        out_file_name = filename.split('/')[-1]
        email_key = out_file_name[0:-6]

    # "Thanks for help." test case
    # if out_file_name != "f01b4829-ba8e-4992-927a-271165eb1a22.i.txt":
        # continue
    
    # if out_file_name != "a88a0b8e-2a69-45c2-8d2a-6af3f88eb6eb.i.txt":
        # continue
        
    if email_key in records_in_pgres:
        continue
    
    '''
    # method 1: print('sender:', next((a['sender'] for a in sender_info if a.get('emailkey') == email_key), None))
    # method 2:
    for d in sender_info:
        if (d['emailkey'] == email_key):
            print(d['sender']) 
            
    method 3: print('sender:', next(map(lambda l : l['sender'], filter(lambda a : a['emailkey'] == email_key, sender_info)), None))
    '''
    
    sender = next(map(lambda l : l['sender'], filter(lambda a : a['emailkey'] == email_key, sender_info)), None)
    
    outCleanFile = out_cleanfile_folder + out_file_name
    
    # if Path(outCleanFile).exists():
        # print("File ", outCleanFile, "found in cdc. Skipping... ")
        # continue

    print(str(cnt) + ") File Read: " + out_file_name)
    
    cnt += 1

    if any([x in content_lower for x in ooo_matches]):
        email_type = "OOO"
        ignore_status = "Y"
        upd_postgres_record(email_key, "P", ignore_status, response_status, email_type)
        continue
        
    if any([x in content_lower for x in volunteer_matches]):
        email_type = "VOLUNTEER"
        ignore_status = "Y"
        upd_postgres_record(email_key, "P", ignore_status, response_status, email_type)
        continue
    
    if any([x in content_lower for x in undelivered_matches]):
        email_type = "UNDELIVERABLE"
        ignore_status = "Y"
        upd_postgres_record(email_key, "P", ignore_status, response_status, email_type)
        continue
    
    cidStripped = re.sub(r'\[cid.*?\]', ' ', fileContent, flags=re.MULTILINE|re.IGNORECASE)

    # Remove Multiple empty lines and remove Multiple Spaces for address match to work with "re."
    trimmedTxt = '\n'.join([line.strip() for line in cidStripped.splitlines()])
    trimmedTxt = re.sub(r' {2,}' , ' ', trimmedTxt)
    
    first_line = trimmedTxt.splitlines()[0]
    if fileContent.count('\n') > 1:
        second_line = trimmedTxt.splitlines()[1]
    else:
        second_line = ""
        
    if first_line.startswith("test") or second_line.startswith("PersonName:TestJ") or sender.find("test") != -1:
        email_type = "TEST"
        ignore_status = 'Y'
        upd_postgres_record(email_key, "P", ignore_status, response_status, email_type)
        continue
    
    # if first_line.startswith(("rds support request ref", "f#", "h#", "o#")):
    if second_line.startswith("PersonName:"):
        email_type = "SUPP_REF"
        eomText, sal_end_found = find_supp_ref_eom(trimmedTxt)
        cleanedTxt, org_list = final_cleanup(eomText, email_type, sal_end_found)
        
    if first_line.startswith("rds support request ref") and not second_line.startswith("PersonName:"):
        email_type = "RE_SUPP_REF"
    
    if email_type != "SUPP_REF":
        # loremStripped = omhStripped.split("Lorem", 1)[0]
        disclaimStripped = trimmedTxt
        
        if "\nDisclaimer" in trimmedTxt or "\nDISCLAIMER" in trimmedTxt:
            disclaimStripped = trimmedTxt.split("\nDisclaimer", 1)[0]
            
        omhStripped = disclaimStripped.split("Original Message Header", 1)[0] 
        omhStripped = omhStripped.split("Original message header", 1)[0]
        # input("B4 find_end_of_message.. Press Enter to continue..")
        
        eomText, email_type, response_status, sal_end_found = find_end_of_message(omhStripped)
        # print('eomText:\n', eomText, 'email_type:', email_type, 'response_status:', response_status, 'sal_end_found:',sal_end_found)
        # x = str(input("B4 final_cleanup.. Press Enter to continue.."))
        cleanedTxt, org_list = final_cleanup(eomText, email_type, sal_end_found)
        # print(cleanedTxt)
        # x = input("Hit return to continue..")
    
    if len(cleanedTxt) < 1:
        ignore_status = "I"
        response_status = ""
        email_type = "THANKS"
        upd_postgres_record(email_key, "P", ignore_status, response_status, email_type)
        continue
    
    # Write to file
    out_clean_file = open(outCleanFile, "w")
    out_clean_file.write(cleanedTxt)
    out_clean_file.flush()
    out_clean_file.close()
    
    if len(cleanedTxt) > 5000 or len(cleanedTxt) < 100:
        out_error_file.write(out_file_name + ", " + str(len(cleanedTxt)) + "\n")
        out_error_file.flush()
        
    # if len(org_list) > 0:
        # org_list_file.write("File: " + out_file_name + "\n" + org_list)
        # org_list_file.flush()

    # Update PostgreSQL database table 
    upd_postgres_record(email_key, "C", ignore_status, response_status, email_type)
    
out_error_file.close()
# org_list_file.close()
cursor.close()
conn.close()